In [3]:
import pandas as pd
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
#sqlContext = SQLContext(sc)
from nltk.corpus import stopwords
import re as re
from pyspark.ml.feature import CountVectorizer , IDF
from pyspark.mllib.linalg import Vector, Vectors
#from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.ml.clustering import LDA, LDAModel
import nltk
import numpy as np
from pyspark.sql import functions as F
from pyspark.sql.functions import size
from pyspark.sql.functions import udf, col
from pyspark.ml.feature import RegexTokenizer,StopWordsRemover
from pyspark.sql.functions import udf, struct
import pyspark.sql.types as T 
import string
import nltk
from pyspark.sql import Row

import os
import sys
from pyspark.sql import SparkSession



In [4]:
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
spark =SparkSession.builder.master("local[1]").appName('SparkProject').getOrCreate() 

21/12/09 00:26:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/09 00:26:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
#data  = spark.read.json(r"C:\Users\EndUser\Desktop\Data\AbhishekData\tweets.json")
#df = spark.read.json("tweets2.csv")
df = spark.read.json("tweets90.json")

In [6]:
df.head()

Row(content='.@VW will introduce heads-up display with augmented-reality in its upcoming #EVs. https://t.co/CINqosgiCD', date='2017-01-31T23:49:34+00:00', latitude=None, longitude=None, tweetid=826578200402616320, userid=1187313746, userlocation='')

In [7]:
df = df.dropDuplicates(['tweetid'])
df = df.na.drop()
df = df.filter("content != ''")

In [8]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def remove_number(tweet):
    tweet = re.sub('([0-9]+)', '', tweet) 
    return tweet

def remove_users(tweet):
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) 
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) 
    return tweet

In [9]:
from pyspark.sql.types import StringType

remove_emoji=udf(remove_emoji)
remove_users=udf(remove_users)
remove_number=udf(remove_number)

df=df.withColumn('content', remove_emoji(df['content']))
df=df.withColumn('content', remove_users(df['content']))
df=df.withColumn('content', remove_number(df['content']))

In [10]:

df = df.withColumn('content', F.regexp_replace('content', r'http\S+', ''))
df = df.withColumn('content', F.regexp_replace('content', '@\w+', ''))
df = df.withColumn('content', F.regexp_replace('content', '#', ''))
df = df.withColumn('content', F.regexp_replace('content', 'RT', ''))
df = df.withColumn('content', F.regexp_replace('content', ':', ''))
df = df.withColumn('content', F.regexp_replace('content', '\n', ''))
df = df.withColumn('content', F.regexp_replace('content', '|', ''))
df = df.withColumn('content', F.regexp_replace('content', '[^\sa-zA-Z0-9]', ''))
df = df.withColumn("content",F.lower('content'))
df = df.withColumn("content",F.rtrim('content'))

In [11]:
df.head()

Row(content=' european fast charging networks announce fast charging alliance evs', date='2017-02-12T19:18:39+00:00', latitude=59.8097794, longitude=10.4914682, tweetid=830858673421443077, userid=1664893472, userlocation='📍Amsterdam & Costa Rica ')

In [12]:
tokenizer = RegexTokenizer().setPattern("[\\W_]+").setMinTokenLength(3).setInputCol("content").setOutputCol("tokens")

In [13]:
df = tokenizer.transform(df)


In [29]:
df.head()

Row(content=' european fast charging networks announce fast charging alliance evs', date='2017-02-12T19:18:39+00:00', latitude=59.8097794, longitude=10.4914682, tweetid=830858673421443077, userid=1664893472, userlocation='📍Amsterdam & Costa Rica ', tokens=['european', 'fast', 'charging', 'networks', 'announce', 'fast', 'charging', 'alliance', 'evs'])

In [14]:
nltk.download('stopwords')
stopwordList = stopwords.words('english')
remover = StopWordsRemover(stopWords=stopwordList)
remover.setInputCol("tokens")
remover.setOutputCol("filtered")

df = remover.transform(df);


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/bigdata/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatization(row):
    #row = [stemmer.stem(lemmatizer.lemmatize(word)) for word in row]
    row = [lemmatizer.lemmatize(word,'v') for word in row]
    return row

[nltk_data] Downloading package wordnet to /home/bigdata/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [16]:
lemmatization = udf(lemmatization)

df=df.withColumn('tokens', lemmatization(df['filtered']))



In [33]:
df.head()

Row(content=' european fast charging networks announce fast charging alliance evs', date='2017-02-12T19:18:39+00:00', latitude=59.8097794, longitude=10.4914682, tweetid=830858673421443077, userid=1664893472, userlocation='📍Amsterdam & Costa Rica ', tokens='[european, fast, charge, network, announce, fast, charge, alliance, evs]', filtered=['european', 'fast', 'charging', 'networks', 'announce', 'fast', 'charging', 'alliance', 'evs'])

In [34]:
#sampling data for manual labelling

In [44]:
df = df.sample(0.17)

In [36]:
df = df.select("tweetid", "content")

In [40]:
from pyspark.sql.functions import lit
df = df.withColumn("polarity", lit(0))
df = df.withColumn("topic", lit(""))

In [1]:
df.head()

NameError: name 'df' is not defined

In [47]:
df.coalesce(1).write.option("header", True).csv('sample.csv')

In [17]:
from textblob import TextBlob

sentiment_polarity = udf(lambda x: TextBlob(x).sentiment[0])
spark.udf.register("sentiment_polarity", sentiment_polarity)

sentiment_subjectivity = udf(lambda x: TextBlob(x).sentiment[1])
spark.udf.register("sentiment_subjectivity", sentiment_subjectivity)

<function __main__.<lambda>(x)>

In [18]:
df = df.withColumn('sentiment_polarity',sentiment_polarity('content').cast('double'))
df.head()

Row(content=' european fast charging networks announce fast charging alliance evs', date='2017-02-12T19:18:39+00:00', latitude=59.8097794, longitude=10.4914682, tweetid=830858673421443077, userid=1664893472, userlocation='📍Amsterdam & Costa Rica ', tokens='[european, fast, charge, network, announce, fast, charge, alliance, evs]', filtered=['european', 'fast', 'charging', 'networks', 'announce', 'fast', 'charging', 'alliance', 'evs'], sentiment_polarity=0.13333333333333333)

In [19]:
df = df.withColumn('sentiment_subjectivity',sentiment_subjectivity('content').cast('double'))
df.head()

Row(content=' european fast charging networks announce fast charging alliance evs', date='2017-02-12T19:18:39+00:00', latitude=59.8097794, longitude=10.4914682, tweetid=830858673421443077, userid=1664893472, userlocation='📍Amsterdam & Costa Rica ', tokens='[european, fast, charge, network, announce, fast, charge, alliance, evs]', filtered=['european', 'fast', 'charging', 'networks', 'announce', 'fast', 'charging', 'alliance', 'evs'], sentiment_polarity=0.13333333333333333, sentiment_subjectivity=0.39999999999999997)